In [1]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
from geopy.geocoders import Nominatim
import folium
from pandas.io.json import json_normalize

print("Libraries have been imported succesfully!")

Libraries have been imported succesfully!


In [2]:
webPage = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M") #Download page
html = webPage.text                                                                       #Get HTML code
tableInit = html.find('<table class="wikitable sortable">')                               #Locate index for beginning of the table
tableFinal = html.find('</table>')                                                        #Locate index for ending of the table
htmlTable = html[tableInit:tableFinal]                                                    #HTML table extracted
table = pd.read_html(htmlTable, header = 0)[0]                                            #From HTML to Pandas data frame

print("Table has been downloaded succesfully!")
table.head()

Table has been downloaded succesfully!


,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [8]:
table["Borough"] = table["Borough"].replace({"Not assigned":np.nan})                  #Convert "Not assigned" from Borough to NaN
table.dropna(inplace = True)                                                          #Remove all rows that have NaN 
table.where(table != "Not assigned", table["Borough"], axis = 0, inplace = True)      #Convert "Not assigned" from Neighbourhood to Borough
joinedRows = table.groupby("Postal Code")["Neighborhood"].apply(lambda x: ", ".join(x)) #Join rows with the same "Postcode" with a comma between
table.drop_duplicates(["Postal Code"],inplace = True)                                    #Remove duplicates so that the joined rows and table have the same shape
df = table.join(joinedRows, on = "Postal Code", lsuffix='_single')                       #Join the new row and the table
df.drop(columns = ["Neighborhood_single"], inplace = True)                           #Drop the column of neighbourhood that is not usefull anymore
df.reset_index(drop = True, inplace = True)                                           #Reset index

print("Data preprocessed!. Shape of table: "+ str(df.shape))
df.head(10)

Data preprocessed!. Shape of table: (103, 3)


,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [25]:
geo = pd.read_csv('/Users/Peter/Downloads/Geospatial_Coordinates.csv')                       
df_final = df.merge(geo, on = "Postal Code") 
df_final.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [27]:
address = 'Toronto'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

C:\Users\Peter\anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.22.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [29]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_final['Latitude'], df_final['Longitude'], df_final['Borough'], df_final['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [30]:

xToronto_data = df_final[df_final['Borough'].str.contains('Toronto')].reset_index(drop=True)
xToronto_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [32]:
map_xtoronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(xToronto_data['Latitude'], xToronto_data['Longitude'], xToronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_xtoronto)  
    
map_xtoronto

In [33]:
CLIENT_ID = '2I0S3UDT4JPCUVSECPX2NUVA1DCGMOJCICB5PMJJIVQNMXHV' # your Foursquare ID
CLIENT_SECRET = 'HR1NF5MW52YXVDV0KWI5G3XFVWXFRHRNOBYRQEUCGKXA10CL' # your Foursquare Secret
VERSION = '20180605'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2I0S3UDT4JPCUVSECPX2NUVA1DCGMOJCICB5PMJJIVQNMXHV
CLIENT_SECRET:HR1NF5MW52YXVDV0KWI5G3XFVWXFRHRNOBYRQEUCGKXA10CL


In [35]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [38]:
LIMIT = 100
xToronto_venues = getNearbyVenues(names=xToronto_data['Neighborhood'],
                                   latitudes=xToronto_data['Latitude'],
                                   longitudes=xToronto_data['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [39]:
xToronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,56,56,56,56,56,56
"Brockton, Parkdale Village, Exhibition Place",24,24,24,24,24,24
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,64,64,64,64,64,64
Christie,17,17,17,17,17,17
Church and Wellesley,75,75,75,75,75,75
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,33,33,33,33,33,33


In [40]:
xToronto_onehot = pd.get_dummies(xToronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
xToronto_onehot['Neighborhood'] = xToronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [xToronto_onehot.columns[-1]] + list(xToronto_onehot.columns[:-1])
xToronto_onehot = xToronto_onehot[fixed_columns]

xToronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:

xToronto_grouped = xToronto_onehot.groupby('Neighborhood').mean().reset_index()
xToronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,Berczy Park,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.017857,0.000000,0.000000,0.000000,0.000000,0.00000
1,"Brockton, Parkdale Village, Exhibition Place",0.041667,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
2,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.0625,0.0625,0.0625,0.125,0.1875,0.125,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
4,Central Bay Street,0.015625,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.015625,0.000000,0.000000,0.015625,0.000000,0.00000
5,Christie,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
6,Church and Wellesley,0.026667,0.013333,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.013333,...,0.013333,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.013333,0.00000
7,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.040000,...,0.000000,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.00000
8,Davisville,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.030303,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
9,Davisville North,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000


In [42]:
num_top_venues = 10

for hood in xToronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = xToronto_grouped[xToronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.05
2                Café  0.04
3  Seafood Restaurant  0.04
4            Beer Bar  0.04
5         Cheese Shop  0.04
6          Restaurant  0.04
7              Bakery  0.04
8        Gourmet Shop  0.02
9    Department Store  0.02


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.12
1     Coffee Shop  0.08
2  Breakfast Spot  0.08
3          Bakery  0.08
4     Yoga Studio  0.04
5    Intersection  0.04
6       Pet Store  0.04
7    Climbing Gym  0.04
8       Nightclub  0.04
9      Restaurant  0.04


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                  venue  freq
0  Gym / Fitness Center  0.06
1         Garden Center  0.06
2                  Park  0.06
3           Pizza Place  0.06
4            Restaurant  0.06
5               Butcher  0.06
6         Burrito Place  0.06
7               Br

In [43]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = xToronto_grouped['Neighborhood']

for ind in np.arange(xToronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(xToronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Café,Cheese Shop,Beer Bar,Seafood Restaurant,Restaurant,Bakery,Breakfast Spot,Irish Pub
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Bakery,Convenience Store,Nightclub,Italian Restaurant,Restaurant,Intersection,Burrito Place
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Comic Shop,Fast Food Restaurant,Farmers Market,Burrito Place,Auto Workshop,Spa,Restaurant,Pizza Place,Butcher
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Terminal,Airport Lounge,Plane,Boat or Ferry,Rental Car Location,Sculpture Garden,Harbor / Marina,Boutique,Airport Gate
4,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Japanese Restaurant,Department Store,Salad Place,Thai Restaurant,Burger Joint,Bubble Tea Shop
5,Christie,Grocery Store,Café,Park,Athletics & Sports,Diner,Restaurant,Candy Store,Italian Restaurant,Nightclub,Coffee Shop
6,Church and Wellesley,Sushi Restaurant,Coffee Shop,Japanese Restaurant,Gay Bar,Restaurant,Hotel,Burger Joint,Pub,Bubble Tea Shop,Yoga Studio
7,"Commerce Court, Victoria Hotel",Coffee Shop,Café,Restaurant,Hotel,Gym,American Restaurant,Seafood Restaurant,Italian Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant
8,Davisville,Pizza Place,Dessert Shop,Sandwich Place,Coffee Shop,Café,Sushi Restaurant,Italian Restaurant,Gym,Restaurant,Farmers Market
9,Davisville North,Sandwich Place,Breakfast Spot,Hotel,Department Store,Convenience Store,Pizza Place,Food & Drink Shop,Gym,Park,Diner


In [44]:
kclusters = 5

xToronto_grouped_clustering = xToronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(xToronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
print(kmeans.labels_[:])

[2 2 2 2 2 2 2 2 2 1 2 2 1 2 2 2 2 2 1 2 4 2 2 2 2 2 4 3 2 2 2 2 2 2 2 0 2
 2 2]


In [46]:
xToronto_merged = xToronto_data
xToronto_merged['Cluster Labels'] = kmeans.labels_
xToronto_merged = xToronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
xToronto_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Restaurant,Café,Theater,Cosmetics Shop,Beer Store
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,2,Coffee Shop,Sushi Restaurant,Diner,Sandwich Place,Bar,Smoothie Shop,Beer Bar,Music Venue,Italian Restaurant,Sculpture Garden
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,2,Clothing Store,Coffee Shop,Bubble Tea Shop,Cosmetics Shop,Japanese Restaurant,Italian Restaurant,Café,Middle Eastern Restaurant,Diner,Pizza Place
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,2,Café,Coffee Shop,American Restaurant,Restaurant,Cocktail Bar,Gastropub,Hotel,Department Store,Clothing Store,Italian Restaurant
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Asian Restaurant,Pub,Trail,Health Food Store,Dim Sum Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner


In [48]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []
for lat, lon, poi, cluster in zip(xToronto_merged['Latitude'], xToronto_merged['Longitude'], xToronto_merged['Neighborhood'], xToronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters